In [53]:
## Pipeline for identifying reads with multiple SNPs matching the alternate allele
## Imports
import pandas as pd
import pysam
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)
import ast
from tqdm import tqdm


INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [49]:
## Create the lookup dictionary from the lookup dataframe, return a dictionary with the chromosome as the key and a set of positions as the value
def get_positions_by_chrom_set(lookup_df):
    positions_by_chrom = {}
    for _, row in lookup_df.iterrows():
        chrom, pos = row['Chrom'], row['Pos']
        if chrom not in positions_by_chrom:
            positions_by_chrom[chrom] = set()
        positions_by_chrom[chrom].add(pos)
    return positions_by_chrom

## Create the lookup dictionary from the lookup dataframe, return a dictionary with the chromosome as the key and a dictionary of positions with ref and alt alleles as the value
def get_positions_by_chrom_dict(lookup_df):
    positions_by_chrom = {}
    for _, row in lookup_df.iterrows():
        chrom, pos, ref, alt = row['Chrom'], row['Pos'], row['Ref'], row['Alt']
        if chrom not in positions_by_chrom:
            positions_by_chrom[chrom] = {}
        positions_by_chrom[chrom][pos] = {'ref': ref, 'alt': alt}
    return positions_by_chrom
    

## Filter the reads to only include reads that cover more than 1 SNP position
def filter_reads_2SNPs(bam_file, positions_by_chrom, filtered_bam_file):
    # Convert positions to a dictionary grouped by chromosome for faster lookup
    

    # Open the BAM file for reading
    bam = pysam.AlignmentFile(bam_file, "rb")

    # Create a new BAM file for writing the filtered reads
    with pysam.AlignmentFile(filtered_bam_file, "wb", template=bam) as out_bam:
        for read in bam:
            # Skip unmapped reads
            if read.is_unmapped:
                continue
            
            # Get all reference positions covered by the read
            reference_positions = read.get_reference_positions(full_length=True)
            
            # Count how many of the positions in the list are covered by this read
            chrom = read.reference_name
            if chrom in positions_by_chrom:
                covered_positions = [pos for pos in reference_positions if pos in positions_by_chrom[chrom]]
                
                # Write the read to the output BAM if it covers more than one position
                if len(covered_positions) > 1:
                    out_bam.write(read)

    # Close the BAM file
    bam.close()

## Extract the information from the filtered BAM file as a dataframe: columns should be read_name,	cigar, chromosome, ref_positions, query_sequence, alts, refs, snp_pos
def extract_info(filtered_bam_file):

    # Open the filtered BAM file for reading
    bam = pysam.AlignmentFile(filtered_bam_file, "rb")

    # Create a list to store the read information
    read_info = []

    # Iterate over the reads in the BAM file
    for read in bam:
        # Get the read information
        read_info.append({
            "read_name": read.query_name,
            "cigar": read.cigarstring,
            "chromosome": read.reference_name,
            "ref_positions": read.get_reference_positions(full_length=True),
            "query_sequence": read.query_sequence,
        })

    # Close the BAM file
    bam.close()

    # Create a DataFrame from the read information
    df = pd.DataFrame(read_info)
    return df

## Add in the base information from the lookup_df
def add_base_info(df, positions_by_chrom):

    def get_alleles(row):
        chrom = row.chromosome
        positions = row.ref_positions

        alts = []
        refs = []
        snp_pos = []

        for pos in positions:
            if chrom in positions_by_chrom and pos in positions_by_chrom[chrom]:
                alts.append(positions_by_chrom[chrom][pos]['alt'])
                refs.append(positions_by_chrom[chrom][pos]['ref'])
                snp_pos.append(pos)
        return pd.Series([alts, refs, snp_pos])

    df["ref_positions"] = df["ref_positions"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    # Apply the function in parallel
    df[['alts', 'refs','snp_pos']] = df.parallel_apply(get_alleles, axis=1)

    return df
        

In [59]:
## Functions for processing the csv files
## Add in the base info
## For each position in snp_pos, find its index in ref_positions
def find_bases(row):
    snp_pos = row.snp_pos
    ref_positions = row.ref_positions
    bases = []
    for pos in snp_pos:
        if pos in ref_positions:
            ind = ref_positions.index(pos) - 1
            bases.append(row.query_sequence[ind])
    return bases

def filter_reads_col(row):
        matches = 0
        for i in range(len(row['bases'])):
            if row['bases'][i] == row['alts'][i]:
                matches += 1
        return matches

def filter_reads_ped(row):
        matches = 0
        for i in range(len(row['bases'])):
            if row['bases'][i] == row['refs'][i]:
                matches += 1
        return matches

def process_file(file_path, ecotype):
     ## Load the dataframe
    df = pd.read_csv(file_path, index_col=None)
    ## Convert ref_positions to a list
    df["ref_positions"] = df["ref_positions"].parallel_apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    df["snp_pos"] = df["snp_pos"].parallel_apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    df["alts"] = df["alts"].parallel_apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    df["refs"] = df["refs"].parallel_apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    ## Add the base information
    df['bases'] = df.parallel_apply(find_bases, axis=1)
    ## Calculate the number of alt matches
    if ecotype == "col":
        df['alt_matches'] = df.parallel_apply(filter_reads_col, axis=1)
    elif ecotype == "ped":
        df['alt_matches'] = df.parallel_apply(filter_reads_ped, axis=1)
    ## Save the dataframe again
    df.to_csv(file_path, index = None)
    ## Delete the dataframe from memory to save space
    del df

In [50]:
## Create the look up files
lookup_df = pd.read_csv("/Users/tomkinsm/reanalysis-mobile-mrna/Data/thieme_snps_from_homograft.csv", index_col=None)
positions_by_chrom_set = get_positions_by_chrom_set(lookup_df)
positions_by_chrom_dict = get_positions_by_chrom_dict(lookup_df)

In [62]:
bam_files = ["C-C-root-FN.bam","C-C-shoot-FN.bam","P-P-root-FN.bam","P-P-shoot-FN.bam","Col-FN-root-1.bam","Col-FN-root-2.bam","Col-FN-root-3.bam","Col-FN-shoot-1.bam","Col-FN-shoot-2.bam","Col-FN-shoot-3.bam","Ped-FN-root-1.bam","Ped-FN-root-2.bam","Ped-FN-root-3.bam","Ped-FN-shoot-1.bam","Ped-FN-shoot-2.bam","Ped-FN-shoot-3.bam"]

In [ ]:
## >2 hours to run
bam_directory = "/Users/tomkinsm/mRNA_analysis/BAM/"

## Code to process the BAM files
for file in tqdm(bam_files):
    bam_file = bam_directory + file
    filtered_bam_file = bam_file.replace(".bam", "_filtered-2SNPs.bam")
    filter_reads_2SNPs(bam_file, positions_by_chrom_set, filtered_bam_file)
    ## Create an index for the filtered BAM file
    pysam.index(filtered_bam_file)
    df = extract_info(filtered_bam_file)

    ## Add the base information to the dataframe
    df = add_base_info(df, positions_by_chrom_dict)

    ## Save the dataframes to csv
    df.to_csv(bam_file.replace(".bam", ".csv"))

    ## Remove the bam_file and dataframes from memory
    del df
    del bam_file


100%|██████████| 4/4 [27:45<00:00, 416.31s/it]


In [ ]:
## 29 minutes to run
## Process the csv files
for rep in tqdm(bam_files):
    if rep.startswith("C"):
        process_file(bam_directory + rep.replace(".bam", ".csv"), "col")
    elif rep.startswith("P"):
        process_file(bam_directory + rep.replace(".bam", ".csv"), "ped")


100%|██████████| 16/16 [29:21<00:00, 110.06s/it] 


In [ ]:
# Function to check if the first value in ref_positions is the same as the first value in snp_pos

# Load the DataFrame
directory_path = "/Users/tomkinsm/mRNA_analysis/BAM/"

for file in tqdm(bam_files):
    rep = file.replace(".bam", ".csv")
    file_path = directory_path + rep
    test_df = pd.read_csv(file_path, index_col=None)

    test_df['snp_pos'] = test_df['snp_pos'].parallel_apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    test_df['ref_positions'] = test_df['ref_positions'].parallel_apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    ## We filter the first and last 2 positions of each read
    threshold_first = 2
    threshold_final = 2

    # Function to check if the first value in ref_positions is more than 3 bases away from the first value in snp_pos
    # and the final value in snp_pos is not within 3 bases of the final position in ref_positions
    def calculate_values(row):
        if row['ref_positions'] and row['snp_pos']:
            if row['ref_positions'][0] is not None and row['snp_pos'][0] is not None and row['ref_positions'][-1] is not None and row['snp_pos'][-1] is not None:
                first_condition = abs(row['ref_positions'][0] - row['snp_pos'][0]) > threshold_first
                final_condition = abs(row['ref_positions'][-1] - row['snp_pos'][-1]) > threshold_final
                return first_condition and final_condition
        return False

    # Apply the function to each row and filter the DataFrame
    filtered_df = test_df[test_df.apply(calculate_values, axis=1)].copy()
    filtered_df['snp_pos'] = filtered_df['snp_pos'].parallel_apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    unique_reads = filtered_df['read_name'].nunique()
    print(f"Number of total reads: {unique_reads}")

    ## Obtain the number of reads where both bases match their respectives alts
    matching_alts = filtered_df[filtered_df.apply(lambda row: row['alt_matches'] == len(row['snp_pos']), axis=1)]
    unique_reads = matching_alts['read_name'].nunique()
    print(f"Number of reads where all SNPs match the alternate allele: {unique_reads}")

    ## Save the data
    filtered_path = file_path.replace(".csv", "_filtered.csv")
    matching_path = file_path.replace(".csv", "_matching.csv")
    filtered_df.to_csv(filtered_path, index=None)
    matching_alts.to_csv(matching_path, index=None)

Number of total reads: 1753179
Number of reads where all SNPs match the alternate allele: 29
Number of total reads: 1977539
Number of reads where all SNPs match the alternate allele: 2
Number of total reads: 610762
Number of reads where all SNPs match the alternate allele: 235
Number of total reads: 366704
Number of reads where all SNPs match the alternate allele: 455
Number of total reads: 832072
Number of reads where all SNPs match the alternate allele: 81
Number of total reads: 834696
Number of reads where all SNPs match the alternate allele: 45
Number of total reads: 891477
Number of reads where all SNPs match the alternate allele: 43
Number of total reads: 838014
Number of reads where all SNPs match the alternate allele: 287
Number of total reads: 1366447
Number of reads where all SNPs match the alternate allele: 108


In [ ]:
## Load in and analyse the data
# Function to generate snp_names
for file in bam_files:
    file_path_matching = bam_directory + rep.replace(".bam", "_matching.csv")
    file_path_filtered = bam_directory + rep.replace(".bam", "_filtered.csv")
    matching = pd.read_csv(file_path_matching, index_col=None)
    filtered = pd.read_csv(file_path_filtered, index_col=None)
    matching_read = matching['read_name'].unique()
    filtered_read = filtered['read_name'].unique()
    # Apply the function to each row
    print(rep)
    print(f"{len(matching_read)} out of {len(filtered_read)}")

In [ ]:
## Get the number of reads with at least 1 SNP matching the alternate allele, but not all SNPs
for file in bam_files:
    file_path = bam_directory + file.replace(".bam", "_filtered_conflicting.csv")
    conflicting = pd.read_csv(file_path, index_col=None)
    print(len(conflicting['read_name'].unique()))

CCRootFN.csv
1675
CCShootFN.csv
1797
ColFNRoot1.csv
674
ColFNRoot2.csv
677
ColFNRoot3.csv
755
ColFNShoot1.csv
721
ColFNShoot2.csv
1243
ColFNShoot3.csv
651
PPRootFN.csv
759
PPShootFN.csv
458
PedFNRoot1.csv
598
PedFNRoot2.csv
538
PedFNRoot3.csv
589
PedFNShoot1.csv
564
PedFNShoot2.csv
735
PedFNShoot3.csv
631
